In [1]:
from EEResnet18 import ResNet, ResidualBlock, data_loader
import numpy as np
import torch
import torch.nn as nn
import torch.backends
import torch.backends.mps
import torch.mps
import torch.nn.functional as F
from tqdm import tqdm

Setting RESNET-18 device as mps


In [2]:
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64, data_model="cifar10")

test_loader = data_loader(data_dir='./data',
                              batch_size=1,
                              test=True, data_model="cifar10")

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Setting TEST device as {device}")

device = torch.device(device)

Files already downloaded and verified
Files already downloaded and verified
50000
Files already downloaded and verified
Setting TEST device as mps


In [3]:
earlyExitLayers = [1, 2, 3, 4]
earlyExitNumber = len(earlyExitLayers)
criterion = nn.CrossEntropyLoss()

def calculate_loss(pred, target):
    cum_loss = []
    accuracy = []
    for i in range(earlyExitNumber + 1):
        cum_loss.append(criterion(pred[i], target))
        _, predicted = torch.max(pred[i].data, 1)
        accuracy.append((predicted == target).sum().item())

    return cum_loss, accuracy

In [4]:
num_classes = 10
num_epochs = 10
# batch_size = 16
learning_rate = 0.01

model = ResNet().to(device)
# model.requires_grad_ = True
# model = ResNet()
model.make_backbone(ResidualBlock, [2, 2, 2, 2], earlyExitLayers, num_classes)

# Loss and optimizer

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 0.001)  

# Train the model
# total_step = len(train_loader)
# print(len(train_loader))
import gc
total_step = len(train_loader)
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} ================")
    # n_batch = (n_sample - 1) // batch_size + 1
    model.train()
    total_sample = 0
    total_loss = total_accuracy = 0
    layers_accuracy = []
    losses = []
    [layers_accuracy.append(0) for i in range(earlyExitNumber + 1)]
    # for j in range(1):
    with tqdm(total=total_step) as bar_step:
        for i, (images, labels) in enumerate(train_loader): 
            # print(i) 
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            # print(outputs[-1].size())
            # _, predicted = torch.max(outputs.data, 1)
            # total_accuracy += (predicted == labels).sum().item()
            # loss = criterion(outputs[-1], labels)
            optimizer.zero_grad()
            loss, accuracy = calculate_loss(outputs, labels)
            for j in range(len(accuracy)):
                layers_accuracy[j] += accuracy[j]
            # print(loss)
            # print(accuracy)
            # total_loss += loss
            # losses.append(float(loss))
            total_sample += labels.size(0)

            # Backward and optimize
            for j in range(len(loss)):
                if j + 1 == len(loss):
                    loss[j].backward()
                else:
                    loss[j].backward(retain_graph = True)
            optimizer.step()
            del images, labels, outputs
            torch.mps.empty_cache() if torch.backends.mps.is_available() else torch.cuda.empty_cache()
            gc.collect()

            if (i + 1) % 3 == 0 or i + 1 == total_step + 1:
                bar_step.set_postfix({
                            'avg_loss': float(total_loss) / (i + 1),
                            'avg_accuracy': (sum(layers_accuracy)/3) / (i + 1),
                            # 'max_abs_gradient': np.max(abs(model.weight.grad))
                        })
                cur_n_batch = i % 3 + 1
                bar_step.update(cur_n_batch)

    # print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, round(np.mean(losses))))
    for p in range(len(layers_accuracy)):
        accuracy1 = 100 * layers_accuracy[p] / total_sample
        print(f'Epoch {epoch+1}: Accuracy of Exit layer {p + 1} = {accuracy1:.2f}%')
        
    # model.eval()
                
        # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs[-1].data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

Epoch 1 ================


  1%|          | 6/704 [00:09<17:47,  1.53s/it, avg_loss=0, avg_accuracy=12.7]


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs[-1].data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))